In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
cd '/home/dfried/projects/onecommon/aaai2020/experiments'

/home/dfried/projects/onecommon/aaai2020/experiments


In [3]:
import models
from models import get_model_type
import data
import utils
from domain import get_domain
from corpora.reference_sentence import ReferenceSentenceCorpus

In [4]:
ls expts/rel3_tsel_ref_dial_model_separate/directions

bad_filtered/
base_512_extremes_asymmetric_learned-pooling_untied_bidirectional_hidden-mention-encoder_rb-lpm-lm-lPm-lPpm_prb-lpm-lm-lPm-lPpm/
base_512_extremes_asymmetric_learned-pooling_untied_bidirectional_hidden-mention-encoder_rb-lpm-lm_prb-lpm-lm/
base_512_extremes_learned-pooling_untied_bidirectional_hidden-mention-encoder/
base_512_extremes_learned-pooling_untied_bidirectional_hidden-mention-encoder_2/
base_512_extremes_learned-pooling_untied_bidirectional_hidden-mention-encoder_3/
base_512_extremes_learned-pooling_untied_bidirectional_hidden-mention-encoder-filtered-separate/
base_512_extremes_learned-pooling_untied_bidirectional_hidden-mention-encoder-filtered-shared/
base_512_extremes_learned-pooling_untied_bidirectional_hidden-mention-encoder_mb-tpmpred-lmpred/
base_512_extremes_learned-pooling_untied_bidirectional_hidden-mention-encoder_mention-start-10/
base_512_extremes_learned-pooling_untied_bidirectional_hidden-mention-encoder_rb-lpm-lm-lPm-lPpm_prb-lpm-lm-lPm-lPpm/
ba

In [5]:
ls expts/rel3_tsel_ref_dial_model_separate/directions/base_512_learned-pooling_untied_bidirectional_hidden-mention-encoder/

 1_best.stdict
 1_best.th
 1_ep-30.stdict
 1_ep-30.th
'eval_dev_cand=40_oracle.out'
'eval_dev_cand=40_speaker-weight=0.1.out'
'eval_dev_cand=40_speaker-weight=0.2.out'
'eval_dev_cand=40_speaker-weight=0.3.out'
'eval_dev_cand=40_speaker-weight=0.4.out'
'eval_dev_cand=40_speaker-weight=0.5.out'
'eval_dev_cand=40_speaker-weight=0.6.out'
'eval_dev_cand=40_speaker-weight=0.7.out'
'eval_dev_cand=40_speaker-weight=0.8.out'
'eval_dev_cand=40_speaker-weight=0.9.out'
'eval_dev_cand=40_speaker-weight=0.out'
'eval_dev_cand=40_speaker-weight=1.0.out'
 eval_dev_l0.out
 eval_dev_l0_ref_annotations.json
 train_fold-1.out


In [6]:
domain = get_domain('one_common')
unk_threshold = 10
model_ty = get_model_type('rnn_reference_model')

ctx_encoder_type = 'rel_attn_encoder_3'

# model_fname = 'rel2_dial_model_separate/hierarchical_only-self_mask-pad_max-pool_rel-2_feed-context-attend_no-word-attention_ablate-properties_1_best.th'

base = 'expts/rel3_tsel_ref_dial_model_separate/directions'

model_fnames = {
    'basic': f'{base}/base_512_learned-pooling_untied_bidirectional_hidden-mention-encoder/1_ep-30.th',
    'encoder-filtered-shared': f'{base}/base_512_extremes_learned-pooling_untied_bidirectional_hidden-mention-encoder-filtered-shared/1_ep-30.th',
    'encoder-filtered-separate': f'{base}/base_512_extremes_learned-pooling_untied_bidirectional_hidden-mention-encoder-filtered-separate/1_ep-30.th',
}

In [7]:
seed = 1
corpus = ReferenceSentenceCorpus(domain, 'data/onecommon',
                            train='train_reference_{}.txt'.format(seed),
                            valid='valid_reference_{}.txt'.format(seed),
                            test='test_reference_{}.txt'.format(seed),
                            freq_cutoff=unk_threshold, verbose=True)

freq cutoff: 10
vocab size: 649
dataset data/onecommon/train_reference_1.txt, total 545056, unks 4302, ratio 0.79%
dataset data/onecommon/valid_reference_1.txt, total 69348, unks 686, ratio 0.99%
dataset data/onecommon/test_reference_1.txt, total 67284, unks 636, ratio 0.95%


In [8]:
def load_json(file):
    import json
    with open(file, 'r') as f:
        return json.load(f)

In [9]:
scenarios1 = load_json('data/onecommon/aaai_train_scenarios.json')
scenarios2 = load_json('data/onecommon/aaai_train_scenarios_2.json')

In [10]:
len(scenarios1)

9000

In [11]:
len(scenarios2)

4500

In [12]:
scenarios1[0].keys()

dict_keys(['kbs', 'shared', 'uuid'])

In [13]:
train_scenarios = {
    scenario['uuid']: scenario
    for scenario in scenarios1
}

In [46]:
all_scenarios = {
    scenario['uuid']: scenario
    for scenario in scenarios1 + scenarios2
}

In [14]:
hit_count = 0
miss_count = 0
for scenario in scenarios2:
    uuid = scenario['uuid']
    if uuid in train_scenarios:
        assert train_scenarios[uuid] == scenario
        hit_count += 1
    else:
        miss_count += 1

In [15]:
print("hits: {}".format(hit_count))
print("misses: {}".format(miss_count))

hits: 0
misses: 4500


In [16]:
models = {
    name: utils.load_model(model_fname, map_location='cpu', prefix_dir=None)
    for name, model_fname in model_fnames.items()
}

/home/dfried/miniconda3/envs/onecommon/lib/python3.6/site-packages/torch/serialization.py:657: SourceChangeWarning: source code of class 'models.rnn_reference_model.HierarchicalRnnReferenceModel' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/home/dfried/miniconda3/envs/onecommon/lib/python3.6/site-packages/torch/serialization.py:657: SourceChangeWarning: source code of class 'models.ctx_encoder.RelationalAttentionContextEncoder3' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/home/dfried/miniconda3/envs/onecommon/lib/python3.6/site-packages/torch/serialization.py:657: SourceChangeWarning: source code of class 'models.atte

In [17]:
batchsize=1
validset, validset_stats = corpus.valid_dataset(batchsize)

100%|██████████████████████████████████████| 1038/1038 [00:01<00:00, 731.36it/s]


In [18]:
validset_stats

{'n': 74030, 'self_n': 37015, 'nonpadn': 69348, 'self_nonpadn': 34674}

In [19]:
import torch

In [20]:
from agent import RnnAgent

In [21]:
from selfplay import make_parser

In [22]:
parser = make_parser()
args = parser.parse_args('--temperature 0.25'.split())
# args = parser.parse_args('--temperature 0.01'.split())

In [23]:
def is_selection(out):
    return '<selection>' in out

In [24]:
# display

In [25]:
from dialog import DialogLogger

In [26]:
from IPython.display import SVG, display, HTML

In [27]:
def display_svgs(svgs):
    no_wrap_div = '<div style="white-space: nowrap">' + ''.join(svgs) + '</div>'
    display(HTML(no_wrap_div))

In [28]:
SVG_SCALE=0.6

In [29]:
def display_attn(scenario, attn, agent_id, name=None):
    attn = attn.flatten().detach().numpy()
    if name is not None:
        print("{}: {}".format(name, attn))
    display_svgs([DialogLogger._scenario_to_svg(scenario, scale=SVG_SCALE)[agent_id], DialogLogger._attention_to_svg(scenario, agent_id, attn, scale=SVG_SCALE)])

In [30]:
# training emulation

In [31]:
trainset, trainset_stats = corpus.train_dataset(bsz=1)

100%|██████████████████████████████████████| 8304/8304 [00:13<00:00, 631.11it/s]


In [32]:
trainset[0].scenario_ids[0] in train_scenarios

True

In [33]:
# ctx: this player's dots
# inpt / tgt: dialogue word indices (tgt are shifted by 1)
# ref_inpt

In [34]:
for ix in range(len(trainset)):
#     ctx, inpts, tgts, ref_inpts, ref_tgts, sel_tgt, \
#     [scenario_id], real_ids, agents, chat_ids, sel_idxs, \
#     lens, rev_idxs, hid_idxs, all_num_markables = trainset[ix]
    
    ctx, inpts, tgts, ref_inpts, ref_tgts, sel_tgt, \
    [scenario_id], real_ids, partner_real_ids, agents, chat_ids, sel_idxs, \
    lens, rev_idxs, hid_idxs, num_markables, \
    is_self, partner_ref_inpts, partner_ref_tgts_our_view, all_partner_num_markables,\
    ref_disagreements, partner_ref_disagreements = trainset[ix]

    if 'C_d0de3d0144df405e998ae718539ff9a4' in chat_ids:
#     if scenario_id == 'S_UoujG1GG6KscmnbN':
        print("found it")
        scenario = train_scenarios[scenario_id]
        break

found it


In [35]:
models.keys()

dict_keys(['basic', 'encoder-filtered-shared', 'encoder-filtered-separate'])

In [36]:
model = models['encoder-filtered-separate']

In [37]:
alice = RnnAgent(model, args)
bob = RnnAgent(model, args)

In [59]:
def force_contexts_and_mentions(index, datasplit=validset, samples=10, sample_temperature=0.25, argmax_temperature=0.005):
    ctx, inpts, tgts, ref_inpts, ref_tgts, sel_tgt, \
    [scenario_id], real_ids, partner_real_ids, agents, chat_ids, sel_idxs, \
    lens, rev_idxs, hid_idxs, num_markables, \
    is_self, partner_ref_inpts, partner_ref_tgts_our_view, all_partner_num_markables,\
    ref_disagreements, partner_ref_disagreements = datasplit[index]
    
    scenario = all_scenarios[scenario_id]
    agent = alice
    agent.feed_context(ctx.flatten())
    agent.real_ids = real_ids
    agent.agent_id = agents[0]

    writer = alice
    conv = []
    speaker = []
    words_left = 5000

    sentence_ix = 0

    assert len(inpts) == len(tgts)

    while sentence_ix < len(inpts):
        words_og = [corpus.word_dict.idx2word[ix] for ix in inpts[sentence_ix].flatten().numpy()]

        if ref_inpts[sentence_ix] is not None:
            dots_mentioned = (ref_tgts[sentence_ix].sum(dim=1) > 0)
            dots_mentioned_per_ref = ref_tgts[sentence_ix]
            this_num_markables = num_markables[sentence_ix]
    #         display_attn(scenario, dots_mentioned.float(), writer.agent_id, name='dots_mentioned')
            for mention_ix in range(dots_mentioned_per_ref.size(1)):
                display_attn(scenario, dots_mentioned_per_ref[:,mention_ix].float(), writer.agent_id, name=f'dots_mentioned_{mention_ix}')
        else:
            dots_mentioned = torch.zeros(1, 7).bool()
            dots_mentioned_per_ref = torch.zeros(1, 0, 7).bool()
            this_num_markables = torch.LongTensor([0])

        if is_self[sentence_ix]:
            for sample_ix in range(samples):
                pred_outs, _, _, _, extra = writer.model.write(
                    writer.ctx, writer.ctx_differences, writer.ctx_h, writer.reader_and_writer_lang_h, 
                    words_left, sample_temperature,
                    dots_mentioned=dots_mentioned,
                    dots_mentioned_per_ref=dots_mentioned_per_ref,
                    num_markables=this_num_markables
                )
                print('sample {}\t{}'.format(sample_ix, ' '.join(writer._decode(pred_outs, writer.model.word_dict))))
            pred_outs, _, _, _, extra = writer.model.write(
                writer.ctx, writer.ctx_differences, writer.ctx_h, writer.reader_and_writer_lang_h, 
                words_left, argmax_temperature,
                dots_mentioned=dots_mentioned,
                dots_mentioned_per_ref=dots_mentioned_per_ref,
                num_markables=this_num_markables
            )
            print('argmax\t{}'.format(' '.join(writer._decode(pred_outs, writer.model.word_dict))))
    #         print(' '.join(words_og))

            out = writer.write(
                max_words=words_left,
                force_words=[words_og],
                start_token=words_og[0],
                dots_mentioned=dots_mentioned,
                dots_mentioned_per_ref=dots_mentioned_per_ref,
                num_markables=this_num_markables,
            )

    #         if writer.extras[-1]['feed_ctx_attn_prob'] is not None:
    #             display_attn(scenario, writer.extras[-1]['feed_ctx_attn_prob'], writer.agent_id, name='feed attention')

    #         if 'word_ctx_attn_prob_mean' in writer.extras[-1]:
    #             display_attn(scenario, writer.extras[-1]['word_ctx_attn_prob_mean'], writer.agent_id, name='word attention mean')

    #         for ix, ix_attn in enumerate(writer.extras[-1]['word_ctx_attn_probs']):
    #             display_attn(scenario, ix_attn, writer.agent_id, name='t {}: {}'.format(ix, out[ix]))

        else:
            out = words_og
            writer.read(words_og,
                        dots_mentioned=dots_mentioned,
                        dots_mentioned_per_ref=dots_mentioned_per_ref,
                        num_markables=this_num_markables)

        print('human\t{}'.format(' '.join(out)))
        print()
        print('-'*40)
        print()
        words_left -= len(out)
        conv.append(out)
        if is_selection(out) or words_left <= 1:
            break
        sentence_ix += 1

In [60]:
force_contexts_and_mentions(1)

dots_mentioned_0: [1. 0. 0. 1. 1. 0. 0.]


dots_mentioned_1: [1. 0. 0. 1. 0. 0. 0.]


dots_mentioned_2: [0. 0. 0. 0. 1. 0. 0.]


sample 0	i have a cluster of 3 dots , two medium gray and one dark grey <eos>
sample 1	i have a cluster of three dots , two are dark grey and one is light gray <eos>
sample 2	i have a cluster of 3 dots . two are dark grey and one is light grey <eos>
sample 3	i have a cluster of three dots . two are darker and one is lighter <eos>
sample 4	i have a cluster of three dots , 2 are same size and one is lighter <eos>
sample 5	i have a cluster of 3 dots , two are black and one light gray <eos>
sample 6	i have a cluster of three dots , the top one is darker than the other two <eos>
sample 7	i have a cluster of 3 dots , two are dark grey and one is light grey <eos>
sample 8	i have a cluster of three dots . two are similar sized and the one on the left is darker <eos>
sample 9	i have a triangle of two medium sized dots close together . one is smaller and darker than the other <eos>
argmax	i have a cluster of 3 dots , two are dark grey and one is light grey <eos>
human	i have a cluster of dots al

sample 0	yes i see that <eos>
sample 1	i see that <eos>
sample 2	i have that <eos>
sample 3	yes i see those dots <eos>
sample 4	i see that <eos>
sample 5	yes i see that <eos>
sample 6	yes i have that <eos>
sample 7	yes i see that <eos>
sample 8	yes i have that <eos>
sample 9	yes i see that <eos>
argmax	yes i see that <eos>
human	yes i think i have that same cluster . which should we select ? <eos>

----------------------------------------

human	THEM: do you a have a medium gray a little <unk> 6 oclock very near the bottom <eos>

----------------------------------------

dots_mentioned_0: [1. 0. 0. 0. 0. 0. 0.]


dots_mentioned_1: [0. 0. 0. 0. 1. 0. 0.]


dots_mentioned_2: [1. 0. 0. 1. 1. 0. 0.]


sample 0	yes . it is on the bottom of the line , but there is a dark gray dot to the left of the other two <eos>
sample 1	i think i have the same one . let 's click the large light grey one . the two dots are the same size . <eos>
sample 2	i have a large dark gray dot in the middle of the circle , there is a smaller light gray dot to the right of them <eos>
sample 3	yes , i see that . click the light gray one of the triangle <selection>
sample 4	yes , i see that , but i have two medium gray dots . the one on the right is slightly larger and slightly lighter <eos>
sample 5	yes , i have that , but i have a medium grey dot , and a large grey dot <eos>
sample 6	yes , i see the same thing . is the smaller one to the right of the other two ? <eos>
sample 7	yes , i think i see that . is there a dark grey dot below them ? <eos>
sample 8	yes , i see the same thing . let 's pick the light grey one in the middle of the triangle <selection>
sample 9	i have the same one , let 's pick the large ligh

dots_mentioned_1: [1. 0. 0. 0. 0. 0. 0.]


sample 0	yes , let 's pick the top one . the black one ? <eos>
sample 1	yes , the one on the right is slightly darker , let 's pick that one . <selection>
sample 2	yes , the one on the right is the darkest . let 's pick that one . <eos>
sample 3	yes , the one on the left is slightly darker . let 's pick that one . <selection>
sample 4	yes , the one on the left is darker and slightly above the bottom ? if so click that one ! <selection>
sample 5	yes , the one on the right is slightly darker , let 's choose that one <eos>
sample 6	yes , let 's pick the top one . the darkest one ? <eos>
sample 7	yes , the one on the left is slightly darker ? let 's pick that one . <eos>
sample 8	yes , the one on the right is slightly larger and darker ? let 's pick that one . <selection>
sample 9	yes , the one on the right is light gray . let 's pick that one <eos>
argmax	yes , the one on the right is slightly darker . let 's pick that one . <selection>
human	yes same size . lets click on the bottom of th

In [62]:
force_contexts_and_mentions(2)

dots_mentioned_0: [0. 1. 0. 0. 0. 0. 0.]


dots_mentioned_1: [0. 0. 0. 0. 0. 0. 1.]


dots_mentioned_2: [0. 1. 0. 0. 0. 0. 0.]


dots_mentioned_3: [0. 0. 0. 0. 1. 0. 0.]


dots_mentioned_4: [0. 1. 0. 0. 0. 0. 0.]


sample 0	i have a large black dot with a smaller dark grey dot above it and a medium gray dot to the left of it <eos>
sample 1	i have a large dark dot with a smaller dark grey dot above it and a medium gray dot to the left of it <eos>
sample 2	i have a large black dot with a smaller dark gray dot above and to the right of it . they are very close together <eos>
sample 3	i have a large black dot with a smaller dark grey dot below it and a medium grey dot to the right of it <eos>
sample 4	i have a large black dot with a small dark grey dot below it and a medium gray dot to the right of it <eos>
sample 5	i have a large black dot with a smaller dark grey dot to the left of it , and a medium sized black dot to the right of it <eos>
sample 6	i have a large black dot with a smaller dark grey dot to its right and a smaller lighter one to its left <eos>
sample 7	i have a large black dot with a smaller dark grey dot above and to the right of it . they are very close together <eos>
sample 8	do yo

sample 0	ok let 's pick the large black dot <selection>
sample 1	ok , lets pick the large black dot <selection>
sample 2	ok , lets choose the large black dot <selection>
sample 3	ok , lets click the large black dot <selection>
sample 4	ok , lets click the large black one <selection>
sample 5	ok , lets pick the large black dot <eos>
sample 6	ok , lets choose the large black dot <selection>
sample 7	ok , lets pick the large black dot <selection>
sample 8	let 's pick the large black dot <selection>
sample 9	ok , lets choose the large black one <selection>
argmax	ok , lets pick the large black dot <selection>
human	okay lets do the large dark one then ? <eos>

----------------------------------------

human	THEM: ok <selection>

----------------------------------------



In [63]:
force_contexts_and_mentions(3)

human	THEM: do you have 4 medium sized dots in a <unk> tight line ? the top one is black <eos>

----------------------------------------

dots_mentioned_0: [1. 0. 0. 0. 1. 0. 0.]


sample 0	yes i do , i do see the 3 dots in a line <eos>
sample 1	yes i see that <eos>
sample 2	yes i have that <eos>
sample 3	yes i do ! i see that <eos>
sample 4	yes i have that <eos>
sample 5	yes i do , i have a cluster of 3 dots that form a triangle <eos>
sample 6	yes i see that <eos>
sample 7	i have a cluster of 3 light grey dots <eos>
sample 8	yes i see that <eos>
sample 9	i have a cluster of 3 light grey dots <eos>
argmax	yes i have that <eos>
human	yes ... the two middle ones are light grey ? <eos>

----------------------------------------

human	THEM: yup <eos>

----------------------------------------

dots_mentioned_0: [1. 0. 0. 0. 1. 0. 0.]


sample 0	ok i see that <eos>
sample 1	ok , i see those dots . <eos>
sample 2	ok , i have that . <eos>
sample 3	i have that <eos>
sample 4	do you have two light grey dots ? <eos>
sample 5	i have that <eos>
sample 6	i have that . <eos>
sample 7	i see that <eos>
sample 8	ok , i see that . <eos>
sample 9	i have that too <eos>
argmax	ok , i have that . <eos>
human	are they different sizes ? <eos>

----------------------------------------

human	THEM: yes the top light one is a little bit smaller <eos>

----------------------------------------

dots_mentioned_0: [1. 0. 0. 0. 0. 0. 0.]


sample 0	yes , let 's choose the top light one <selection>
sample 1	yes , let 's pick the top one <selection>
sample 2	yes , lets choose the top one <selection>
sample 3	yes , let 's choose the top one <selection>
sample 4	yes , let 's choose the top one <selection>
sample 5	yes , let 's pick the top one <selection>
sample 6	yes , let 's pick the top one <selection>
sample 7	yes , let 's choose the top one <selection>
sample 8	yes , let 's pick the top one <selection>
sample 9	yes , let 's pick the top one <selection>
argmax	yes , let 's choose the top one <selection>
human	my top one is a light smaller too <eos>

----------------------------------------

human	THEM: ok click the black dot at the top <selection>

----------------------------------------



In [64]:
force_contexts_and_mentions(4)

human	THEM: hello i see 2 <unk> of dots <eos>

----------------------------------------

dots_mentioned_0: [0. 0. 0. 1. 1. 1. 1.]


dots_mentioned_1: [0. 0. 0. 1. 0. 0. 0.]


dots_mentioned_2: [0. 0. 0. 0. 1. 1. 1.]


dots_mentioned_3: [0. 0. 0. 1. 0. 0. 0.]


sample 0	i have a cluster of 4 dots , two are medium grey and one is dark grey and the other is black <eos>
sample 1	i have a cluster of 3 dots , two are dark grey and one medium grey . the other is larger and lighter . <eos>
sample 2	i have a cluster of 3 dots . two are medium gray , one is dark grey and one is large and darker . <eos>
sample 3	i have a cluster of 3 dots . two are dark grey and one light grey . <eos>
sample 4	i have a cluster of 3 dots , two are dark grey and one is small and light grey . the one on the right is large and black <eos>
sample 5	i have a triangle of three dots . two are dark grey and one light grey . <eos>
sample 6	i have a cluster of three dots , two are dark and one light <eos>
sample 7	i have a cluster of 4 dots , the lowest is the lightest and largest , the two are darker and to the right of the other one <eos>
sample 8	i have a cluster of 3 dots . two are dark grey and one light grey <eos>
sample 9	i have a cluster of 3 dots . one is light grey and 

sample 0	i have that <eos>
sample 1	i have those <eos>
sample 2	i have that <eos>
sample 3	i see that <eos>
sample 4	i see that <eos>
sample 5	i have that <eos>
sample 6	i see that <eos>
sample 7	i have that <eos>
sample 8	i see that <eos>
sample 9	i have that <eos>
argmax	i see that <eos>
human	yes , that 's the cluster , which one did you pick ? <eos>

----------------------------------------

human	THEM: i did the middle grey one <selection>

----------------------------------------



In [65]:
force_contexts_and_mentions(5)

dots_mentioned_0: [0. 1. 0. 1. 0. 1. 1.]


sample 0	i have a cluster of 4 dots . <eos>
sample 1	i have a cluster of 4 dots in a triangle <eos>
sample 2	do you see a cluster of dots ? <eos>
sample 3	i see a cluster of 3 dots that form a triangle <eos>
sample 4	i have 4 dots in a line <eos>
sample 5	do you see a cluster of dots ? <eos>
sample 6	i have a cluster of three dots that form a triangle <eos>
sample 7	i see a cluster of 4 dots <eos>
sample 8	i have a cluster of 4 dots <eos>
sample 9	i have a cluster of 4 dots in a triangle <eos>
argmax	i have a cluster of 4 dots <eos>
human	do you see a group of 4 in a <unk> square ? <eos>

----------------------------------------

human	THEM: the smallest one is on the top ? <eos>

----------------------------------------

dots_mentioned_0: [0. 1. 0. 0. 0. 0. 0.]


sample 0	yes , lets choose the smallest one <eos>
sample 1	yes , let 's pick that one <eos>
sample 2	yes , lets pick the smallest dot <eos>
sample 3	yes , lets pick the top one <eos>
sample 4	yes , lets pick the one on the right <eos>
sample 5	yes . let 's choose the top one <eos>
sample 6	yes , let 's pick the smallest one <eos>
sample 7	yes , pick the smallest ? <eos>
sample 8	yes , let 's choose the top one <eos>
sample 9	yes , pick the smallest one ? <eos>
argmax	yes , let 's pick the top one <eos>
human	yes ! shall we choose the small one ? <eos>

----------------------------------------

human	THEM: yes ! <selection>

----------------------------------------



In [66]:
force_contexts_and_mentions(6)

human	THEM: small black dot aligned vertically with medium light grey dot <eos>

----------------------------------------

dots_mentioned_0: [0. 0. 0. 0. 1. 0. 0.]


dots_mentioned_1: [0. 0. 0. 0. 1. 0. 0.]


sample 0	yes , i see it . let 's pick that small black dot . <eos>
sample 1	yes , i have that one . let 's pick that small black dot . <selection>
sample 2	yes , it is the smallest and black <eos>
sample 3	yes , i have that one . let 's pick that one . <eos>
sample 4	yes i see it . let 's pick that small black dot <eos>
sample 5	yes , i see it . let 's pick that tiny black dot . <selection>
sample 6	yes , it is the smallest and black <eos>
sample 7	yes , i see it . let 's choose that tiny black dot <selection>
sample 8	yes , i see it . let 's pick that tiny black dot . <eos>
sample 9	yes , the small black dot is on the right . let 's choose that one <eos>
argmax	yes , i see it . let 's pick that tiny black dot . <selection>
human	i i think i have that . selecting the small black dot . <eos>

----------------------------------------

human	THEM: is the small black dot slight to the right ? <eos>

----------------------------------------

sample 0	yes <eos>
sample 1	yes <selection>
sampl

In [67]:
force_contexts_and_mentions(7)

dots_mentioned_0: [0. 0. 1. 0. 1. 0. 1.]


dots_mentioned_1: [0. 0. 0. 0. 0. 0. 1.]


dots_mentioned_2: [0. 0. 0. 0. 1. 0. 0.]


dots_mentioned_3: [0. 0. 1. 0. 0. 0. 0.]


sample 0	i have a cluster of 3 dots . one is black and one medium grey , one small and light grey . the one on the right is the darkest and slightly larger . <eos>
sample 1	i have a cluster of 3 dots . one is black and one medium grey , one small , light grey . the other is a dark grey <eos>
sample 2	i have a line of 3 dots . the top one is black and smaller , the other is a small light grey . the top one is black <eos>
sample 3	i have a cluster of 3 dots . two are black , one light grey , one dark grey . <eos>
sample 4	i have a cluster of 3 dots , two are black and one light gray . a small grey dot <eos>
sample 5	i have a cluster of 3 dots . one is black and one light gray . the other is small and black . <eos>
sample 6	i have a cluster of 3 dots , two are black , one is light grey , one is black <eos>
sample 7	i have a cluster of 3 dots . one is black and one light grey . the other two are black and the other one is larger and darker <eos>
sample 8	i have a cluster of 3 dots . two ar

In [68]:
force_contexts_and_mentions(8)

dots_mentioned_0: [0. 0. 1. 0. 0. 1. 0.]


dots_mentioned_1: [0. 0. 0. 0. 0. 1. 0.]


sample 0	i see two dots that are similar in size and color . they are both medium grey <eos>
sample 1	i have two dots very close together . they are both light grey <eos>
sample 2	i have two dots very close together . they are both similar shades of grey <eos>
sample 3	i have two dots very close together . they are both medium grey <eos>
sample 4	do you see two dots close to each other ? <eos>
sample 5	i see two dots very close together . they are both similar shades of grey <eos>
sample 6	i have two dots that are similar in size and color . they are both medium grey . <eos>
sample 7	i have two dots very close together . they are both similar shades of grey <eos>
sample 8	i have two dots very close together . they are both similar shades of grey . <eos>
sample 9	i see two dots very close together . they are both medium grey <eos>
argmax	i have two dots very close together . they are both similar shades of grey <eos>
human	two dots same color , almost vertical , lower one is slightly sm

In [69]:
force_contexts_and_mentions(9)

human	THEM: do you see a tiny grey dot above and to the left of a larger , slightly darker one ? <eos>

----------------------------------------

dots_mentioned_0: [1. 0. 0. 0. 0. 0. 0.]


dots_mentioned_1: [1. 1. 1. 1. 1. 1. 1.]


dots_mentioned_2: [1. 0. 0. 0. 0. 0. 0.]


sample 0	yes , i have that . is there a small dark grey dot to the right of the other dots ? <eos>
sample 1	i have a tiny light gray dot that is to the right and above a large dark grey dot <eos>
sample 2	no . i have a small light gray dot in the middle of 4 other dots . i have a cluster of 4 dots <eos>
sample 3	i have a small gray dot by itself and a very large light grey dot to the right of those <eos>
sample 4	yes i see that . is there a small dot of the same size to the right of it ? <eos>
sample 5	yes i have that one . is there a large dark grey dot to the right of them ? <eos>
sample 6	i have a small light grey dot , all of my dots are large and light gray . do you have a large dark grey dot in the middle of the circle ? <eos>
sample 7	i see a small grey dot on the right side of the circle . there is a large dark grey dot to the right of it <eos>
sample 8	i have a small grey dot that is the lightest of all the rest are the darkest and medium size <eos>
sample 9	i have one small l

dots_mentioned_1: [1. 0. 0. 0. 0. 0. 0.]


dots_mentioned_2: [0. 0. 0. 0. 0. 1. 0.]


dots_mentioned_3: [1. 0. 0. 0. 0. 0. 0.]


sample 0	i have two dots that are almost in a vertical line . the one on the left is larger and lighter than the one on the right <eos>
sample 1	i have a light gray dot that is on the left side of the circle . there is a medium grey dot above it and to the left a little <eos>
sample 2	i have a light gray dot , it is a small dark gray dot . and there is a larger dark grey dot to the right of it <eos>
sample 3	i have a light gray dot , and a large grey dot to the right of it . i do have a medium black dot <eos>
sample 4	i have a small light dot at the top of the circle , there is a dark grey dot above it and a medium grey to the right of it <eos>
sample 5	yes , i have a small light gray dot , and a large dark grey dot , and a medium gray dot that is a little larger . <eos>
sample 6	i have one that is medium gray , and to the right of it is a smaller dark dot . and above that is a small grey dot <eos>
sample 7	i have a small light grey dot that is to the left and above a black dot and a m

In [70]:
force_contexts_and_mentions(10)

human	THEM: lone medium dark black dot near 2 oclock <eos>

----------------------------------------

dots_mentioned_0: [0. 0. 1. 0. 0. 0. 0.]


sample 0	i have a large black dot at the top <eos>
sample 1	i have a large black dot at the bottom <eos>
sample 2	yes i do , let 's click that one . <eos>
sample 3	yes , lets pick that one <selection>
sample 4	i have a large black dot at the top <eos>
sample 5	yes , lets choose it <selection>
sample 6	yes , i have that one <eos>
sample 7	i have a medium black dot at the top <eos>
sample 8	i have a large black dot at the top <eos>
sample 9	yes , i have that one <eos>
argmax	i have a large black dot at the top <eos>
human	ah yes it is a little more center for me but <unk> very alone ? <eos>

----------------------------------------

human	THEM: ok lets try it <selection>

----------------------------------------



In [71]:
force_contexts_and_mentions(11)

human	THEM: dark large dot on 3 o'clock <eos>

----------------------------------------

dots_mentioned_0: [0. 0. 0. 0. 1. 0. 0.]


sample 0	i have a large light grey dot at the bottom <eos>
sample 1	i have a large light gray dot at the bottom <eos>
sample 2	i have a large gray dot at the top <eos>
sample 3	i have a large light gray dot on the left side <eos>
sample 4	i have a large light gray dot at the bottom <eos>
sample 5	i have a large light grey dot at the top <eos>
sample 6	yes , it is in the middle <eos>
sample 7	yes , the light gray is to the right and below ? <eos>
sample 8	i have a large light grey dot at the top <eos>
sample 9	i see a large light grey dot on the right side <eos>
argmax	i have a large light gray dot at the top <eos>
human	large gray dot in the center <eos>

----------------------------------------

human	THEM: nope light large gray dot on 7 o'clock with smaller gray dot more toward the border also on 7 oclock <eos>

----------------------------------------

dots_mentioned_0: [0. 0. 1. 0. 0. 0. 0.]


sample 0	yes , i have that one <eos>
sample 1	i have a small dark dot on the left side <eos>
sample 2	i see that one <eos>
sample 3	i have a small dark dot at the bottom <eos>
sample 4	i have a small dark dot at the top <eos>
sample 5	i have a small black dot <eos>
sample 6	i have a small black dot at the top of the circle <eos>
sample 7	i have a small black dot <eos>
sample 8	i have a small dark dot at 10 o clock <eos>
sample 9	yes i do , i think i have that one <eos>
argmax	i have a small black dot at the top <eos>
human	no . small dark dot at about 2 o'clock <eos>

----------------------------------------

human	THEM: i see tiny dark dot but about 1 oclock <eos>

----------------------------------------

dots_mentioned_0: [0. 0. 1. 0. 0. 0. 0.]


sample 0	i have a small dark dot on the right <eos>
sample 1	i have a small black dot <eos>
sample 2	i have a small black dot <eos>
sample 3	i have a small black dot <eos>
sample 4	i do n't , i have a small dark dot on the right side <eos>
sample 5	i have a small dark dot on the right <eos>
sample 6	i have a small black dot <eos>
sample 7	is it near the edge <eos>
sample 8	i have a small black dot at 3 o clock <eos>
sample 9	i have a small dark dot on the left <eos>
argmax	i have a small black dot <eos>
human	okay yeah . i think that is the same one <eos>

----------------------------------------

human	THEM: let <unk> it <selection>

----------------------------------------



In [72]:
import json
from utils import ContextGenerator

In [43]:
ctx_gen = ContextGenerator('data/onecommon/shared_4.txt')

In [44]:
with open('data/onecommon/shared_4.json', "r") as f:
    scenario_list = json.load(f)
selfplay_scenarios = {scenario['uuid']: scenario for scenario in scenario_list}

In [45]:
scenario = None
for ix, ([scenario_id], (ctx1, ctx2), (ids1, ids2)) in enumerate(ctx_gen.iter()):
#     if scenario_id == 'S_11J8n40u4HRVduwi':
#         print('found it')
#         scenario = scenarios[scenario_id]
#         break
    if ix == 2:
        scenario = selfplay_scenarios[scenario_id]
        print(scenario_id)
        break

S_xM5bXi3lMqXHKKgP


In [46]:
import random

In [ ]:
# selfplay emulation
scenario = selfplay_scenarios[scenario_id]
for agent, agent_id, ctx, real_ids in zip([alice, bob], [0, 1], [ctx1, ctx2], [ids1, ids2]):
    agent.feed_context(ctx)
    agent.real_ids = real_ids
    agent.agent_id = agent_id
    
dot_mention_samples = 1
utt_samples = 10
    
writer, reader = [alice, bob]
conv = []
speaker = []
words_left = 5000

num_mentions = 2
num_dots_to_mention = 3

sample_temperature = 0.25
argmax_temperature = 0.005

def make_dots_mentioned_per_ref():
#     # fixed
#     return torch.BoolTensor([[1,1,0,0,0,0,0], [1,0,0,0,0,0,0]]).unsqueeze(0)
#     return torch.BoolTensor([[1,1,0,0,0,0,0]]).unsqueeze(0)

    # random initial and drop
    mentioned = torch.zeros(1,num_mentions,7).bool()
    dots_mentioned = []
    for _ in range(num_dots_to_mention):
        dot_ix = torch.randint(7, (1,))
        mentioned[:,0,dot_ix] = 1
        dots_mentioned.append(dot_ix)
        
    for mention_ix in range(1, num_mentions):
        mentioned[:,mention_ix,random.choice(dots_mentioned)] = 1
    return mentioned
    

    # random 
#     mentioned = torch.zeros(1,num_mentions,7).bool()
#     for mention_ix in range(num_mentions):
#         for _ in range(num_dots_to_mention):
#             dot_ix = torch.randint(7, (1,))
#             mentioned[:,mention_ix,dot_ix] = 1
#     return mentioned

display_context = True

while True:
    dots_mentioned_per_ref = make_dots_mentioned_per_ref()
    dots_mentioned = dots_mentioned_per_ref.max(1).values
    
    num_markables=torch.LongTensor([dots_mentioned_per_ref.size(1)])
    if display_context:
        for mention_ix in range(dots_mentioned_per_ref.size(1)):
            display_attn(scenario, dots_mentioned_per_ref[:,mention_ix].float(), writer.agent_id, name=f'dots_mentioned_{mention_ix}')
    for us in range(utt_samples):
        outs, _, _, _, extra = writer.model.write(
            writer.ctx,
            writer.ctx_differences,
            writer.ctx_h,
            writer.reader_and_writer_lang_h,
            words_left,
            sample_temperature,
            dots_mentioned=dots_mentioned,
            dots_mentioned_per_ref=dots_mentioned_per_ref,
            num_markables=num_markables,
        )
        print('agent {}; sample {}\t{}'.format(writer.agent_id, us, ' '.join(writer._decode(outs, writer.model.word_dict))))
    print()
        
    num_markables=torch.LongTensor([num_mentions])
    if display_context:
        for mention_ix in range(dots_mentioned_per_ref.size(1)):
            display_attn(scenario, dots_mentioned_per_ref[:,mention_ix].float(), writer.agent_id, name=f'dots_mentioned_{mention_ix}')
    out = writer.write(
        max_words=words_left,
        dots_mentioned=dots_mentioned,
        dots_mentioned_per_ref=dots_mentioned_per_ref,
        num_markables=num_markables,
        temperature_override=argmax_temperature,
    )
    print('agent {}; argmax\t{}'.format(writer.agent_id, ' '.join(out)))
    print()
    
#     if display_context and writer.extras[-1].get('feed_ctx_attn_prob', None) is not None:
#         display_attn(scenario, writer.extras[-1]['feed_ctx_attn_prob'], writer.agent_id, name='feed attention')
    
#     if display_context and writer.extras[-1].get('word_ctx_attn_prob_mean', None) is not None:
#         display_attn(scenario, writer.extras[-1]['word_ctx_attn_prob_mean'], writer.agent_id, name='word attention mean')
        
#     for ix, ix_attn in enumerate(writer.extras[-1]['word_ctx_attn_probs']):
#         display_attn(scenario, ix_attn, writer.agent_id, name='t {}: {}'.format(ix, out[ix]))
        
    print('{}\t{}'.format(writer.agent_id, ' '.join(out)))
    print('-'*60)
        
    words_left -= len(out)
    conv.append(out)
    speaker.append(writer.agent_id)
    reader.read(out)
    if is_selection(out) or words_left <= 1:
        break
    writer, reader = reader, writer

NameError: name 'selfplay_scenarios' is not defined

> <ipython-input-41-e274016a0281>(2)<module>()
      1 # selfplay emulation
----> 2 scenario = selfplay_scenarios[scenario_id]
      3 for agent, agent_id, ctx, real_ids in zip([alice, bob], [0, 1], [ctx1, ctx2], [ids1, ids2]):
      4     agent.feed_context(ctx)
      5     agent.real_ids = real_ids

--KeyboardInterrupt--
